# Reconstruction Sandbox
This notebook is a test-bed for regularization and reconstruction methods

In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

# Load motiondeblur module and Dataset class
import libwallerlab.projects.motiondeblur as md
from libwallerlab.utilities.io import Dataset, isDataset

# Platform imports
import os, glob

# Debugging imports
import llops as yp
import matplotlib.pyplot as plt

yp.config.setDefaultBackend('arrayfire')

## Load Data

In [2]:
# Define user for path setting
dataset_path = '/Users/zfphil/datasets/motiondeblur/'

# Define which dataset to use
dataset_type = 'coded'
dataset_label =  'res'

# Find files in this directory
folder_list = glob.glob(os.path.join(dataset_path, '*/'))
dataset_list = [folder for folder in folder_list if isDataset(folder)]

# Filter datasets in directory
filtered_dataset_list = [folder_name for folder_name in folder_list if (dataset_type in folder_name) and (dataset_label in folder_name)]
assert not len(filtered_dataset_list) > 1, "More than one dataset with criterion found!"
assert not len(filtered_dataset_list) == 0, "No dataset with criterion found!"

# Create dataset object (loads metadata)
dataset = Dataset(filtered_dataset_list[0])

# Force type to be motiondeblur
dataset.metadata.type = 'motiondeblur'

# Select green channel
dataset.channel_mask = [0]

# Preprocess dataset (md-specific)
md.preprocess(dataset)

Loading 1 frames...


/Users/zfphil/.conda/lib/python3.7/site-packages/arrayfire/array.py:157: ComplexWarning: Casting complex values to real discards the imaginary part
  c_val = c_double_t(val)


# Create Reconstruction Object

In [16]:
# Set position segment
# dataset.position_segment_indicies = [3]
dataset.frame_mask = [20, 21, 22]

# Create recon object
recon = md.recon.Reconstruction(dataset, alpha_blend_distance=0, pad_mode='edge')

# Apply frame-dependent position offset
# recon.applyFrameDependentOffset(offset=26)

# Apply frame lateral offset
# recon.applySegmentDependentOffset()

# Normalize measurements
recon.normalize_measurements()

# Register Measurements
# offsets = recon.register_measurements(method='xc', preprocess_methods=['normalize', 'highpass'], axis=1, debug=False, write_results=True)

[1.0, 1.0285789, 1.0989231]

## Sweep Offset

In [12]:
recon_list = []
for _ in yp.display.progressBar(range(0, 100)):
    recon.applyFrameDependentOffset(offset=1)
    recon.reconstruct(iteration_count=-1, step_size=1, mode='global', reg_types={'l2': 1e-3, 'tv': 1e-3})
    recon_list.append(recon.object_recovered)
#     recon.applyFrameDependentOffset(offset=-offset)
# # recon.show()
# plt.figure()
# plt.imshow(yp.real(recon.object_recovered)[200:500, 1200:1700])

# yp.display.listPlotScroll(recon_list)

In [ ]:
yp.display.listPlotScroll(recon_list)